In [ ]:
# File paths
tvseries_path = "../data/TVSeries.txt" 
episodes_path = "../data/Episodes.txt" 
userwatched_path = "../data/UserWatched.txt"
output_part1 = "./output1" 
output_part2 = "./output2" 

In [ ]:
################ 
##### Part 1
################

In [ ]:
# Load Episodes.txt: each line is TVSID,SeasonNumber,EpisodeNumber,Title
episodes_rdd = sc.textFile(episodes_path)

In [ ]:
# Map to key=TVSID, value=SeasonNumber
def TVSID_SeasonNumber(line):
    fields=line.split(",")
    TVSID=fields[0]
    SeasonNumber=int(fields[1])

    return( (TVSID,SeasonNumber))

episodes_seasons_rdd = episodes_rdd.map(TVSID_SeasonNumber)

In [ ]:
# Reduce by key to find the maximum SeasonNumber for each TVSID.
# That is equal to the number of seasons if there are not missing "season numbers"
# Result: key=TVSID, value=max_seasons
max_seasons_rdd = episodes_seasons_rdd.reduceByKey(lambda a, b: max(a, b)).cache()

In [ ]:
#max_seasons_rdd.collect()

In [ ]:
# A long TV series has more than 10 seasons
# Filter TV series with max_seasons > 10
# Result: key=TVSID, value=max_seasons (only for long series)
long_series_rdd = max_seasons_rdd.filter(lambda x: x[1] > 10)

In [ ]:
# Load TVSeries.txt: each line is TVSID,Title,TVGenre
tvseries_rdd = sc.textFile(tvseries_path)

def TVSID_TVGenre(line):
    fields=line.split(",")
    TVSID=fields[0]
    TVGenre=fields[2]

    return( (TVSID,TVGenre))

# Map to key=TVSID, value=TVGenre
tvseries_genre_rdd = tvseries_rdd.map(TVSID_TVGenre)

In [ ]:
# Join long_series_rdd with tvseries_genre_rdd to associate long series with their genres
# Join result: key=TVSID, value=(max_seasons, TVGenre)
# Map to key=TVGenre, value=1 for counting the number of long TV series for each TVGenre
long_series_genre_rdd = long_series_rdd.join(tvseries_genre_rdd)\
                                .map(lambda x: (x[1][1], 1))

In [ ]:
# Count the number of long series per genre
# Reduce by key to sum the 1s for each TVGenre
# Result: key=TVGenre, value=count_of_long_series
genre_count_rdd = long_series_genre_rdd.reduceByKey(lambda a, b: a + b)

In [ ]:
#genre_count_rdd.collect()

In [ ]:
genre_count_rdd.saveAsTextFile(output_part1)

In [ ]:
################ 
##### Part 2
################

In [ ]:
# Load UserWatched.txt: each line is Username,StartTimestamp,TVSID,SeasonNumber,EpisodeNumber
userwatched_rdd = sc.textFile(userwatched_path)

In [ ]:
# Filter for episodes where SeasonNumber=1 and EpisodeNumber=1 (user started watching)
def Season1_Episode1(line):
    fields=line.split(',')
    SeasonNum=fields[3]
    EpisodeNumber=fields[4]

    return SeasonNum=='1' and EpisodeNumber=='1'
    


started_watching_rdd = userwatched_rdd.filter(Season1_Episode1)

In [ ]:
#started_watching_rdd.collect()

In [ ]:
# Map to key=Username, value=TVSID and remove duplicates
# Distinct ensures each (Username, TVSID) pair is unique, even if watched multiple times
# Result: key=Username, value=TVSID
def UserTVSID(line):
    fields=line.split(',')
    user=fields[0]
    TVSID=fields[2]

    return (TVSID, user)

started_watching_TVSID_userrdd = started_watching_rdd.map(UserTVSID)\
                                            .distinct()

In [ ]:
#started_watching_TVSID_userrdd.collect()

In [ ]:
# Join started_watching_TVSID_userrdd with max_seasons_rdd to get the number of seasons for each started series
# Join result: key=TVSID, value=(Username, max_seasons)
# Map to key=Username, value=(TVSID, max_seasons)
user_series_rdd = started_watching_TVSID_userrdd.join(max_seasons_rdd) \
                                  .map(lambda x: (x[1][0], (x[0], x[1][1]))).cache()

In [ ]:
#user_series_rdd.sortBy(lambda p:p[0]).collect()

In [ ]:
# Find the maximum number of seasons per user
# Map to key=Username, value=max_seasons
# Reduce by key to get the maximum seasons for each user
# Result: key=Username, value=max_max_seasons
max_user_seasons_rdd = user_series_rdd.map(lambda x: (x[0], x[1][1])) \
                                    .reduceByKey(lambda a, b: max(a, b))

In [ ]:
#max_user_seasons_rdd.collect()

In [ ]:
# Join to find all series matching the maximum seasons per user
# Before applying join map both input RDDs to pairs with Username, max_seasons as key

# Join result: key=Username, value=((TVSID, max_seasons), max_max_seasons)
combined_rdd = user_series_rdd.map(lambda p: ((p[0],p[1][1]), p[1][0]) )\
                                .join(max_user_seasons_rdd.map(lambda p: ((p[0],p[1]), None)))

In [ ]:
#combined_rdd.collect()

In [ ]:
#Format output as (Username,TVSID)
output_part2_rdd=combined_rdd.map(lambda p: (p[0][0], p[1][0]))

In [ ]:
#output_part2_rdd.collect()

In [ ]:
output_part2_rdd.saveAsTextFile(output_part2)
